In [ ]:
"# ΓΙΝΕΤΑΙ ΧΡΗΣΗ FREE PASS ΚΑΙ ΟΧΙ ΠΟΛΙΤΙΚΗ ΓΙΑ ΚΑΘΥΣΤΕΡΗΜΕΝΕΣ ΑΣΚΗΣΕΙΣ, ΕΙΧΑΜΕ ΧΡΗΣΙΜΟΠΟΙΗΣΕΙ 2 FREE PASS ΚΑΙ ΤΩΡΑ ΑΛΛΑ 2."

"# ΑΣΚΗΣΗ 4\n",
"Βασίλης Πουλος, 2805  \n",
"Κωστάκης Ελευθέριος-Παναγιώτης, 2741"

In [1]:
import pandas as pd
import numpy as np
import json
import time 

dataset_path = '../yelp-dataset/' # Change this path to match your local dataset folder path

business_filename = 'yelp_academic_dataset_business.json'
users_filename = 'yelp_academic_dataset_user.json'
review_filename = 'yelp_academic_dataset_review.json'

In [2]:
# Loading total businesses with at least 50 reviews
business_list = []
with open(dataset_path + business_filename, 'r', encoding="utf8") as buisness_file:
    for line in buisness_file:
        json_dict = json.loads(line)
        if json_dict['review_count'] >= 50:
            business_list.append([ json_dict['business_id'], 
                                   json_dict['name'], 
                                   json_dict['review_count']]) 

In [3]:
business_list = np.array(business_list)
business_list

array([['oiAlXZPIFm2nBCt0DHLu_Q', 'Green World Cleaners', '81'],
       ['x3Po6tJGb729u_HJPY6UCA', 'Glendale Glitters', '52'],
       ['Mmd5WDFq9hHcQ3uClngGjQ', "Irene's Tap Room", '79'],
       ...,
       ['fNil19SUfPAPnLQrYnFrGQ', 'Cheyenne West Animal Hospital', '124'],
       ['JjcJVqhZXhP4tvOhg3fnag', 'Water Heater Pros', '217'],
       ['SYa2j1boLF8DcGVOYfHPcA', 'Five Guys', '97']], dtype='<U64')

In [4]:
# Businesses that have at least 50 reviews
B = set(business_list[:,0])
len(B)

31942

In [5]:
# User, Business, Rating (u, b, r)
start = time.time()

reviews_list = []
with open(dataset_path + review_filename, 'r', encoding="utf8") as reviews_file:
    for line in reviews_file:
        json_dict = json.loads(line)
        if json_dict['business_id'] in B:
            reviews_list.append([json_dict['user_id'], json_dict['business_id'], json_dict['stars']])

reviews_list = np.array(reviews_list)       
stop = time.time()
print("Read file in {:.3f} mins".format((stop-start)/60))

Read file in 1.215 mins


In [6]:
# Conting unique users that commented at least 20 times on businesses from B set

users = reviews_list[:,0]
users, counts = np.unique(users, return_counts=True)
unique_users = []
for i in range(len(users)):
    if counts[i] >= 20:
        unique_users.append(users[i])
unique_users = np.array(unique_users)
len(unique_users)

40225

In [7]:
# convert to set for efficiency
U = set(unique_users)

In [8]:
# load friends information
start = time.time()

list_users = []
with open(dataset_path + users_filename, 'r', encoding="utf8") as users_file:
    for line in users_file:
        json_dict = json.loads(line)
        if json_dict['user_id'] in U:
            list_users.append([ json_dict['user_id'], 
                                json_dict['friends'].replace(',', '').split()])
stop = time.time()
print("Read file in {:.3f} mins".format((stop-start)/60))

Read file in 0.382 mins


## Creating G graph

In [9]:
import networkx as nx

G = nx.Graph()
for user in list_users:
    for friend in user[1]: # user friendlist
        if friend in U: # don't add users that aren't in U.
            G.add_edge(user[0], friend)

In [10]:
largest_cc = max(nx.connected_components(G), key=len)
CC_max = nx.subgraph(G, largest_cc)

### Is max cc the same as the whole graph? (Should be 0)

In [11]:
len(G) - len(CC_max)

168

In [12]:
len(G)

31983

In [13]:
len(CC_max.nodes())

31815

## Pruning

In [14]:
user_set = set(CC_max.nodes())

In [15]:
print(reviews_list)
print("length: {}".format(len(reviews_list)))

[['OwjRMXRC0KyPrIlcjaXeFQ' '-MhfebM0QIsKt87iDN-FNw' '2.0']
 ['V34qejxNsCbcgD8C0HVk-Q' 'HQl28KMwrEKHqhFrrDqVNQ' '5.0']
 ['ofKDkJKXSKZXu5xJNGiiBQ' '5JxlZaqCnk1MnbgRirs40Q' '1.0']
 ...
 ['Scmyz7MK4TbXXYcaLZxIxQ' 'kDCyqlYcstqnoqnfBRS5Og' '5.0']
 ['lBuAACBEThaQHQGMzAlKpg' 'VKVDDHKtsdrnigeIf9S8RA' '3.0']
 ['fiA6ztHPONUkmX6yKIXyHg' '2SbyRgHWuWNlq18eHAx95Q' '5.0']]
length: 5852610


In [16]:
indexes_to_delete = []

for index, user in enumerate(reviews_list[:,0]):
    if user not in user_set:
        indexes_to_delete.append(index)

for index, user in enumerate(reviews_list[:,1]):
    if user not in B:
        indexes_to_delete.append(index)

reviews_pruned = np.delete(reviews_list, indexes_to_delete, 0)
print("{} indexes deleted".format(len(indexes_to_delete)))

4175563 indexes deleted


## Results from pruned table

In [17]:
print("Users: {}".format(len(np.unique(reviews_pruned[:,0]))))
print("Businesses: {}".format(len(np.unique(reviews_pruned[:,1]))))
print("Reviews: {}".format(len(reviews_pruned)))

Users: 31815
Businesses: 31900
Reviews: 1677047


# Part 2

In [18]:
UBR_df = pd.DataFrame(reviews_pruned, columns = ['user_id', 'business_id', 'rating'])
UBR_df

,user_id,business_id,rating
0,V34qejxNsCbcgD8C0HVk-Q,HQl28KMwrEKHqhFrrDqVNQ,5.0
1,UgMW8bLE0QMJDCkQ1Ax5Mg,IS4cv902ykd8wj1TR0N3-A,4.0
2,jOERvhmK6_lo_XGUBPws_w,Ws8V970-mQt2X9CwCuT5zw,4.0
3,A0j21z2Q1HGic7jW6e9h7A,9Jo1pu0y2zU6ktiwQm6gNA,4.0
4,TZQSUDDcA4ek5gBd6BzcjA,qUWqjjjfpB2-4P3He5rsKw,4.0
...,...,...,...
1677042,OmBHPdD3UhTeO3UHq6kBjA,lvOOJmJ4y0BLi31Muq3a8w,2.0
1677043,bH3Uc0YNlnKubj8cT4nQ0w,H8qpFitIesth86zqR4cwYg,5.0
1677044,_g6P8H3-qfbz1FxbffS68g,IsoLzudHC50oJLiEWpwV-w,3.0
1677045,Scmyz7MK4TbXXYcaLZxIxQ,kDCyqlYcstqnoqnfBRS5Og,5.0


In [19]:
import math

sample_data = UBR_df.sample(frac=0.05)
for index in sample_data.index:
    UBR_df.at[index, 'rating'] = 0

print("Sample data should have {} rows \n".format(math.floor(len(UBR_df)*0.05)))
print(len(sample_data))

Sample data should have 83852 rows 

83852


In [20]:
# Create the numpy array, unique_users and unique_business lists
UBR = UBR_df.to_numpy()
unique_users = np.unique(UBR[:,0])
unique_business = np.unique(UBR[:,1])

In [21]:
UBR

array([['V34qejxNsCbcgD8C0HVk-Q', 'HQl28KMwrEKHqhFrrDqVNQ', '5.0'],
       ['UgMW8bLE0QMJDCkQ1Ax5Mg', 'IS4cv902ykd8wj1TR0N3-A', '4.0'],
       ['jOERvhmK6_lo_XGUBPws_w', 'Ws8V970-mQt2X9CwCuT5zw', '4.0'],
       ...,
       ['_g6P8H3-qfbz1FxbffS68g', 'IsoLzudHC50oJLiEWpwV-w', '3.0'],
       ['Scmyz7MK4TbXXYcaLZxIxQ', 'kDCyqlYcstqnoqnfBRS5Og', '5.0'],
       ['fiA6ztHPONUkmX6yKIXyHg', '2SbyRgHWuWNlq18eHAx95Q', 0]],
      dtype=object)

In [22]:
UBR[:,0][0]

'V34qejxNsCbcgD8C0HVk-Q'

In [35]:
sparse_array = np.zeros((len(unique_users), len(unique_business)))
for i in range(len(unique_users)):
    business_index = np.where(unique_users[i] == UBR[:,0]) 
    
    users_ratings = list(UBR[business_index[0]][:,2])
    business_ids = UBR[business_index[0]][:,1]
    sparse_row = np.zeros(len(unique_business))
    for j in range(len(unique_business)):
        if unique_business[j] in business_ids:
            sparse_row[j] = users_ratings.pop(0)
    sparse_array[i] = sparse_row

In [36]:
sparse_array

array([[0., 0., 5., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [38]:
sparse_array.shape

(31815, 31900)